# 1. Import and Install Dependencies

In [1]:
# !pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python mediapipe scikit-learn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints Using MP Holistics

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [8]:
result_test = extract_keypoints(results)

In [9]:
np.save('0', result_test)

In [10]:
np.load('0.npy')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

# 4. Setup Folders For Collection

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

#Actions that we try to detect
actions = np.array(['pinch-fingers spread','multiple-select','box-select', 'scale-uniform', 'scale-X-axis', 'swipe left-to-right', 'pause', 'delete', 'copy', 'cut'])

# 30 videos worth of data
no_sequences = 30

# videos are going to be 30 frames in length
sequence_length = 30

In [19]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values For Training And Testing

In [20]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW Loop
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
    
                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW
                # Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'collecting frames for {} video Number {}'.format(action, sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.waitKey(2000)
                    
                else:
                    cv2.putText(image, 'collecting frames for {} video Number {}'.format(action, sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                

                # Show to screen
                cv2.imshow('OpenCV Feed', image)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
        # Wait for user to press 'q' to move to the next class
        print(f'Ready to collect next class: {action}')
        while True:
            ret, frame = cap.read()
            cv2.imshow('OpenCV Feed', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                print("Moving to the next class")
                break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [21]:
# cap.release()
# cv2.destroyAllWindows()


# 6. Preprocess Data and Create Labels and Features

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label: num for num, label in enumerate(actions)}

In [14]:
label_map

{'pinch-fingers spread': 0,
 'multiple-select': 1,
 'box-select': 2,
 'scale-uniform': 3,
 'scale-X-axis': 4,
 'swipe left-to-right': 5,
 'pause': 6,
 'delete': 7,
 'copy': 8,
 'cut': 9}

In [15]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
                          
                          

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(300, 30, 126)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)



# 7. Build and Train LSTM Neural Network

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [44]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)



In [45]:
model = Sequential()
model.add(LSTM(32, return_sequences = True, activation = 'tanh', input_shape = (30, 126), kernel_regularizer=l2(0.001)))

model.add(LSTM(64, return_sequences = True, activation = 'tanh', kernel_regularizer=l2(0.001)))

model.add(LSTM(32, return_sequences = False, activation = 'tanh', kernel_regularizer=l2(0.001)))

model.add(Dense(32, activation = 'relu', kernel_regularizer=l2(0.001)))
 
model.add(Dense(16, activation = 'relu', kernel_regularizer=l2(0.001)))
model.add(Dense(actions.shape[0], activation='softmax'))

In [46]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [47]:
model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[tb_callback])

Epoch 1/200
8/8 [==============================] - 8s 253ms/step - loss: 2.5806 - categorical_accuracy: 0.1360 - val_loss: 2.5375 - val_categorical_accuracy: 0.1053
Epoch 2/200
8/8 [==============================] - 0s 39ms/step - loss: 2.4446 - categorical_accuracy: 0.1754 - val_loss: 2.3780 - val_categorical_accuracy: 0.1754
Epoch 3/200
8/8 [==============================] - 0s 39ms/step - loss: 2.2948 - categorical_accuracy: 0.2149 - val_loss: 2.2819 - val_categorical_accuracy: 0.1754
Epoch 4/200
8/8 [==============================] - 0s 39ms/step - loss: 2.2077 - categorical_accuracy: 0.1886 - val_loss: 2.2280 - val_categorical_accuracy: 0.1228
Epoch 5/200
8/8 [==============================] - 0s 38ms/step - loss: 2.1466 - categorical_accuracy: 0.2193 - val_loss: 2.2068 - val_categorical_accuracy: 0.1404
Epoch 6/200
8/8 [==============================] - 0s 48ms/step - loss: 2.1112 - categorical_accuracy: 0.2237 - val_loss: 2.1743 - val_categorical_accuracy: 0.1579
Epoch 7/200
8/8

Epoch 51/200
8/8 [==============================] - 0s 40ms/step - loss: 1.0204 - categorical_accuracy: 0.6491 - val_loss: 1.4214 - val_categorical_accuracy: 0.5789
Epoch 52/200
8/8 [==============================] - 0s 39ms/step - loss: 0.9592 - categorical_accuracy: 0.6404 - val_loss: 1.2640 - val_categorical_accuracy: 0.5789
Epoch 53/200
8/8 [==============================] - 0s 40ms/step - loss: 0.9398 - categorical_accuracy: 0.6623 - val_loss: 1.1472 - val_categorical_accuracy: 0.6667
Epoch 54/200
8/8 [==============================] - 0s 42ms/step - loss: 1.0590 - categorical_accuracy: 0.6184 - val_loss: 1.0599 - val_categorical_accuracy: 0.6842
Epoch 55/200
8/8 [==============================] - 0s 40ms/step - loss: 0.9288 - categorical_accuracy: 0.6579 - val_loss: 1.1801 - val_categorical_accuracy: 0.6316
Epoch 56/200
8/8 [==============================] - 0s 39ms/step - loss: 0.9436 - categorical_accuracy: 0.6404 - val_loss: 1.2030 - val_categorical_accuracy: 0.5439
Epoch 57/2

Epoch 101/200
8/8 [==============================] - 0s 39ms/step - loss: 0.4238 - categorical_accuracy: 0.8553 - val_loss: 0.7165 - val_categorical_accuracy: 0.8246
Epoch 102/200
8/8 [==============================] - 0s 38ms/step - loss: 0.3875 - categorical_accuracy: 0.9211 - val_loss: 0.8326 - val_categorical_accuracy: 0.7895
Epoch 103/200
8/8 [==============================] - 0s 37ms/step - loss: 0.4138 - categorical_accuracy: 0.8947 - val_loss: 0.7033 - val_categorical_accuracy: 0.8246
Epoch 104/200
8/8 [==============================] - 0s 40ms/step - loss: 0.4245 - categorical_accuracy: 0.9079 - val_loss: 0.6448 - val_categorical_accuracy: 0.8772
Epoch 105/200
8/8 [==============================] - 0s 43ms/step - loss: 0.3906 - categorical_accuracy: 0.8684 - val_loss: 0.7574 - val_categorical_accuracy: 0.8246
Epoch 106/200
8/8 [==============================] - 0s 44ms/step - loss: 0.4179 - categorical_accuracy: 0.9079 - val_loss: 0.7999 - val_categorical_accuracy: 0.7895
Epoc

8/8 [==============================] - 0s 37ms/step - loss: 0.2645 - categorical_accuracy: 0.9518 - val_loss: 0.6184 - val_categorical_accuracy: 0.9123
Epoch 151/200
8/8 [==============================] - 0s 37ms/step - loss: 0.2379 - categorical_accuracy: 0.9737 - val_loss: 0.5846 - val_categorical_accuracy: 0.8772
Epoch 152/200
8/8 [==============================] - 0s 42ms/step - loss: 0.2293 - categorical_accuracy: 0.9825 - val_loss: 1.0157 - val_categorical_accuracy: 0.7544
Epoch 153/200
8/8 [==============================] - 0s 39ms/step - loss: 0.6234 - categorical_accuracy: 0.8596 - val_loss: 0.6650 - val_categorical_accuracy: 0.8772
Epoch 154/200
8/8 [==============================] - 0s 41ms/step - loss: 0.5954 - categorical_accuracy: 0.8509 - val_loss: 0.7937 - val_categorical_accuracy: 0.8070
Epoch 155/200
8/8 [==============================] - 0s 38ms/step - loss: 0.4076 - categorical_accuracy: 0.8860 - val_loss: 0.4749 - val_categorical_accuracy: 0.8421
Epoch 156/200
8/8 

8/8 [==============================] - 0s 42ms/step - loss: 0.1432 - categorical_accuracy: 1.0000 - val_loss: 0.2762 - val_categorical_accuracy: 0.9649
Epoch 200/200
8/8 [==============================] - 0s 42ms/step - loss: 0.1427 - categorical_accuracy: 1.0000 - val_loss: 0.2765 - val_categorical_accuracy: 0.9649


In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 32)            20352     
                                                                 
 lstm_4 (LSTM)               (None, 30, 64)            24832     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 10)                170       
                                                                 
Total params: 59,354
Trainable params: 59,354
Non-trai


# 8. Make Predictions

In [49]:
res = model.predict(X_test)

In [50]:
actions[np.argmax(res[8])]

'multiple-select'

In [51]:
actions[np.argmax(y_test[8])]

'multiple-select'


# 9. Save Weights

In [52]:
model.save('action.h5')

In [87]:
# del model
# ## model built, compile

In [53]:
# model.load_weights('action.h5')


# 10. Evaluation using Confusion Matrix and Acuuracy

In [54]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [55]:
yhat = model.predict(X_train)

In [56]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [57]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[256,   0],
        [  1,  28]],

       [[256,   0],
        [  0,  29]],

       [[257,   0],
        [  0,  28]],

       [[257,   0],
        [  0,  28]],

       [[255,   0],
        [  0,  30]],

       [[257,   0],
        [  1,  27]],

       [[257,   0],
        [  0,  28]],

       [[257,   1],
        [  0,  27]],

       [[257,   0],
        [  0,  28]],

       [[254,   1],
        [  0,  30]]], dtype=int64)

In [63]:
accuracy_score(ytrue, yhat)

0.9929824561403509

# 11. Test in Real Time

In [59]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16), (245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame
            
        



In [62]:
import cv2
import numpy as np
import mediapipe as mp

# Load the holistic model and other necessary setups from MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.4

# Load the model (assuming it's already loaded in your environment)
# model = your_loaded_model

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process the frame with MediaPipe Holistic
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks with the specific styling
        draw_styled_landmarks(image, results)
        
        # Prediction logic with 30 frames sequence
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            action = actions[np.argmax(res)]
            predictions.append(np.argmax(res))

            # Visualization logic
            if np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold:
                if len(sentence) > 0 and actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(action)
                elif len(sentence) == 0:
                    sentence.append(action)

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Visualize the probabilities
                image = prob_viz(res, actions, image, colors)

                # Find bounding box coordinates for landmarks
                hand_landmarks = results.left_hand_landmarks or results.right_hand_landmarks
                if hand_landmarks:
                    x_min = min([lm.x for lm in hand_landmarks.landmark])
                    y_min = min([lm.y for lm in hand_landmarks.landmark])
                    x_max = max([lm.x for lm in hand_landmarks.landmark])
                    y_max = max([lm.y for lm in hand_landmarks.landmark])
                    
                    x_min = int(x_min * image.shape[1])
                    x_max = int(x_max * image.shape[1])
                    y_min = int(y_min * image.shape[0])
                    y_max = int(y_max * image.shape[0])

                    # Draw rectangle and text
                    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 0), 2)
                    cv2.putText(image, action, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Show to screen
        cv2.imshow('Hand Gesture Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [69]:
res[np.argmax(res)]

0.9991365

In [70]:
res

array([3.0463088e-10, 4.2883339e-06, 9.9913651e-01, 5.9893267e-04,
       2.2498886e-04, 1.5232219e-08, 7.0362785e-06, 2.6412341e-05,
       3.9119354e-07, 1.5393367e-06], dtype=float32)